<a href="https://colab.research.google.com/github/zzhining/python_ml_dl2/blob/main/3%EC%9E%A5/3_2_5_dataset_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 기본 데이터 탐색 함수

In [ ]:
import pandas as pd
import numpy as np

# 예시 데이터 생성 (전자상거래 주문 데이터)
order_data = pd.DataFrame({
    'order_id': ['ORD001', 'ORD002', 'ORD003', 'ORD004', 'ORD005', 'ORD006', 'ORD007', 'ORD008'],
    'customer_id': ['C001', 'C002', 'C001', 'C003', 'C002', 'C004', 'C001', 'C003'],
    'product_category': ['전자제품', '의류', '전자제품', '도서', '의류', '전자제품', '도서', '의류'],
    'order_amount': [150000, 45000, 320000, 25000, 67000, 89000, 35000, 52000],
    'customer_age': [28, 35, 28, 42, 35, 31, 28, 42],
    'region': ['서울', '부산', '서울', '대구', '부산', '서울', '서울', '대구'],
    'order_date': pd.to_datetime(['2024-01-15', '2024-01-16', '2024-01-18', '2024-01-20',
                                  '2024-01-22', '2024-01-25', '2024-01-28', '2024-01-30'])
})

display(order_data)

In [ ]:
# 기본 데이터 정보 탐색
print("=== 데이터프레임 기본 정보 ===")
print(f"데이터 크기: {order_data.shape}")
print(f"컬럼명: {list(order_data.columns)}")

print("\n=== 데이터 타입 정보 ===")
print(order_data.dtypes)

print("\n=== 기술통계량 ===")
print(order_data.describe())

print("\n=== 카테고리별 주문 빈도 ===")
print(order_data['product_category'].value_counts())

print("\n=== 지역별 분포 ===")
print(order_data['region'].value_counts())

print("\n=== 결측치 확인 ===")
print(order_data.isnull().sum())

# 2. 기술통계량 및 데이터 요약

In [ ]:
# 주문금액에 대한 다양한 통계량 계산
amount_stats = order_data['order_amount'].agg(['mean', 'median', 'std', 'min', 'max', 'count'])
print("=== 주문금액 통계량 ===")
print(amount_stats)

# 분위수 계산
print("\n=== 주문금액 분위수 ===")
quantiles = order_data['order_amount'].quantile([0.25, 0.5, 0.75, 0.9])
print(quantiles)

# 고객 연령과 주문금액 간 상관관계
print("\n=== 수치형 변수 간 상관관계 ===")
numeric_corr = order_data[['customer_age', 'order_amount']].corr()
print(numeric_corr)

# 카테고리별 고유값 개수
print("\n=== 각 컬럼별 고유값 개수 ===")
print(order_data.nunique())

# 3. 피봇 테이블 (pivot_table)

In [ ]:
# 지역별, 카테고리별 평균 주문금액 피봇테이블
region_category_pivot = pd.pivot_table(
    order_data,
    values='order_amount',
    index='region',
    columns='product_category',
    aggfunc='mean',
    fill_value=0
)

print("=== 지역별 카테고리별 평균 주문금액 ===")
print(region_category_pivot)

In [ ]:
# 다중 집계 함수 적용
multi_agg_pivot = pd.pivot_table(
    order_data,
    values='order_amount',
    index='region',
    columns='product_category',
    aggfunc=['mean', 'count', 'sum'],
    fill_value=0,
    margins=True  # 총합 행/열 추가
)

print("=== 다중 집계 함수 피봇테이블 (일부) ===")
print(multi_agg_pivot['mean'])  # 평균값만 출력

In [ ]:
# 고객 연령대별, 지역별 주문 건수
order_data['age_group'] = pd.cut(order_data['customer_age'],
                                bins=[0, 30, 40, 50],
                                labels=['20대', '30대', '40대'])

age_region_pivot = pd.pivot_table(
    order_data,
    values='order_id',
    index='age_group',
    columns='region',
    aggfunc='count',
    fill_value=0
)

print("=== 연령대별 지역별 주문 건수 ===")
print(age_region_pivot)

# 4. 그룹별 집계 연산 (groupby)

In [ ]:
# 고객별 주문 통계
customer_stats = order_data.groupby('customer_id').agg({
    'order_amount': ['count', 'sum', 'mean', 'max'],
    'order_id': 'count'
}).round(2)

display(customer_stats)

In [ ]:
# 지역별 상세 분석
region_analysis = order_data.groupby('region').agg({
    'order_amount': ['count', 'sum', 'mean', 'std'],
    'customer_age': 'mean',
    'customer_id': 'nunique'  # 고유 고객 수
}).round(2)

display(region_analysis)

In [ ]:
# 상품 카테고리별 월별 주문 트렌드
order_data['order_month'] = order_data['order_date'].dt.month
monthly_category = order_data.groupby(['order_month', 'product_category']).agg({
    'order_amount': ['count', 'sum'],
    'customer_id': 'nunique'
})

print("=== 월별 카테고리별 주문 현황 ===")
print(monthly_category)

# 그룹별 크기와 기술통계
print("=== 지역별 그룹 크기 ===")
print(order_data.groupby('region').size())

In [ ]:
print("=== 카테고리별 주문금액 기술통계 ===")
category_describe = order_data.groupby('product_category')['order_amount'].describe()
print(category_describe.round(2))

# 5. 멀티레벨 인덱스와 고급 집계

In [ ]:
# 지역별, 카테고리별, 연령대별 다차원 분석
multi_group_analysis = order_data.groupby(['region', 'product_category', 'age_group']).agg({
    'order_amount': ['count', 'sum', 'mean'],
    'customer_id': 'nunique'
}).round(2)

print("=== 다차원 그룹별 분석 ===")
print(multi_group_analysis.head(10))


In [ ]:
# 멀티인덱스를 컬럼으로 변환
unstacked_data = order_data.groupby(['region', 'product_category'])['order_amount'].sum().unstack(fill_value=0)
print("\n=== 지역-카테고리 행렬 (unstack) ===")
print(unstacked_data)

In [ ]:
# 고급 집계: 사용자 정의 함수
def coefficient_of_variation(x):
    """변동계수 계산 (표준편차/평균)"""
    return x.std() / x.mean() if x.mean() != 0 else 0

advanced_stats = order_data.groupby('region')['order_amount'].agg([
    'mean',
    'std',
    coefficient_of_variation,
    lambda x: x.quantile(0.75) - x.quantile(0.25)  # IQR
]).round(3)

advanced_stats.columns = ['평균', '표준편차', '변동계수', 'IQR']
print("\n=== 지역별 고급 통계량 ===")
print(advanced_stats)

In [ ]:
# 조건부 집계
def conditional_mean(group):
    """고액 주문(10만원 이상)의 평균"""
    high_value = group[group['order_amount'] >= 100000]
    return high_value['order_amount'].mean() if len(high_value) > 0 else 0

conditional_stats = order_data.groupby('region').apply(conditional_mean)
print("\n=== 지역별 고액주문 평균금액 ===")
print(conditional_stats.fillna(0).astype(int))

In [ ]:
# 집계 결과의 순위 계산
region_total = order_data.groupby('region')['order_amount'].sum().sort_values(ascending=False)
region_rank = region_total.rank(ascending=False, method='dense')

print("\n=== 지역별 총 주문금액 순위 ===")
result_df = pd.DataFrame({
    '총주문금액': region_total,
    '순위': region_rank.astype(int)
})
print(result_df)